# Ray Serve implementation of a simulated data governance server

It logs the ids of records observed in a data processing pipeline.

In [1]:
import sys, time, json, requests

In [2]:
from ray import serve
import ray

In [3]:
from data_governance_server import DataGovernanceSystem, Record, init_service

In [4]:
port = 8000

In [5]:
ray.init('auto')

2020-05-16 11:16:20,909	WARNING worker.py:809 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.


{'node_ip_address': '192.168.1.149',
 'raylet_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:52525',
 'object_store_address': '/tmp/ray/session_2020-05-16_11-15-44_430322_38090/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-05-16_11-15-44_430322_38090/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-05-16_11-15-44_430322_38090'}

In [6]:
serve.init(http_port=port)

In [7]:
print(f'Click here to open the Ray Dashboard: http://{ray.get_webui_url()}')

Click here to open the Ray Dashboard: http://localhost:8265


In [8]:
init_service()

(pid=38105) 2020-05-16 11:16:21,465	INFO master.py:122 -- Starting router with name 'SERVE_ROUTER_ACTOR'
(pid=38105) 2020-05-16 11:16:21,468	INFO master.py:143 -- Starting HTTP proxy with name 'SERVE_PROXY_ACTOR'
(pid=38105) 2020-05-16 11:16:21,472	INFO master.py:168 -- Starting metric monitor with name 'SERVE_METRIC_MONITOR_ACTOR'
(pid=38105) 2020-05-16 11:16:21,486	INFO master.py:483 -- Registering route /log to endpoint log with methods ['PUT'].
(pid=38105) 2020-05-16 11:16:21,544	INFO master.py:483 -- Registering route /ids to endpoint ids with methods ['GET'].
(pid=38101) INFO:     Started server process [38101]
(pid=38101) INFO:     Waiting for application startup.
(pid=38101) INFO:     Application startup complete.
(pid=38105) 2020-05-16 11:16:21,575	INFO master.py:483 -- Registering route /count to endpoint count with methods ['GET'].
(pid=38105) 2020-05-16 11:16:22,204	INFO master.py:483 -- Registering route /reset to endpoint reset with methods ['PUT', 'GET'].
(pid=38105) 202

'dgs'

In [9]:
def do_test(port, num_records=10, timeout=1.0):
    """Try out the server."""
    records = [Record(i, f'data for record {i}') for i in range(num_records)] # sample "records"

    address = f'http://127.0.0.1:{port}'
    print(f'Putting {num_records} records... to {address}')
    for record in records:
        response = requests.put(f'{address}/log?id={record.record_id}', timeout=timeout)
        print(f'log response = {response.json()}')

    count = requests.get(f'{address}/count', timeout=timeout)
    print(f'count:  {count.json()}')

    ids = requests.get(f'{address}/ids', timeout=timeout)
    print(f'ids:    {ids.json()}')
    
    up_time = requests.get(f'{address}/up_time', timeout=timeout)
    print(f'uptime: {up_time.json()}')

In [10]:
do_test(port)

Putting 10 records... to http://127.0.0.1:8000
log response = {'message': 'sent async log request for 0'}
log response = {'message': 'sent async log request for 1'}
log response = {'message': 'sent async log request for 2'}
log response = {'message': 'sent async log request for 3'}
log response = {'message': 'sent async log request for 4'}
log response = {'message': 'sent async log request for 5'}
log response = {'message': 'sent async log request for 6'}
log response = {'message': 'sent async log request for 7'}
log response = {'message': 'sent async log request for 8'}
log response = {'message': 'sent async log request for 9'}
count:  {'count': 10}
ids:    {'ids': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']}
uptime: {'up_time': 4.391387939453125}


Reset (removed the logged ids)

In [11]:
requests.put(f'http://127.0.0.1:{port}/reset')

<Response [200]>

In [12]:
count = requests.get(f'http://127.0.0.1:{port}/count')
print(f'count now = {count.json()}')

count now = {'count': 0}
(pid=38357) 2020-05-16 11:22:05,832	ERROR worker.py:421 -- SystemExit was raised from the worker
(pid=38357) Traceback (most recent call last):
(pid=38357)   File "python/ray/_raylet.pyx", line 531, in ray._raylet.task_execution_handler
(pid=38357)   File "python/ray/_raylet.pyx", line 419, in ray._raylet.execute_task
(pid=38357)   File "python/ray/_raylet.pyx", line 459, in ray._raylet.execute_task
(pid=38357)   File "python/ray/_raylet.pyx", line 462, in ray._raylet.execute_task
(pid=38357)   File "python/ray/_raylet.pyx", line 463, in ray._raylet.execute_task
(pid=38357)   File "<ipython-input-25-8bf5106e4d29>", line 5, in ray_log_record
(pid=38357)   File "/Users/deanwampler/anaconda3/envs/spark/lib/python3.7/site-packages/ray/worker.py", line 1508, in get
(pid=38357)     values = worker.get_objects(object_ids, timeout=timeout)
(pid=38357)   File "/Users/deanwampler/anaconda3/envs/spark/lib/python3.7/site-packages/ray/worker.py", line 306, in get_objects
(p